In [2]:
import numpy as np
%run utilityFunc.ipynb

In [3]:
# heuristic function 0
# a very simple heuristice which only return 0 if the state is the goal state and return 1 otherwise

def simple_heuristic(puzzle, goalState):
    goal1 = goalState[0]
    goal2 = goalState[1]
    if np.array_equal(puzzle, goal1)or np.array_equal(puzzle, goal2):
        return 0
    else:
        return 1

In [4]:
puzzle = np.array([[1,2,3,4],[5,6,7,0]])
goal1 = [np.array([[1,2,3,4],[5,6,7,0]]),np.array([[1,3,5,7],[2,4,6,0]])]
print(simple_heuristic(puzzle,goal1))

0


In [5]:
# heuristic function 1
# Enhanced Manhattan distance, if the empty tile is not at corners use the usual Manhattan distance
# if the empty tile is at the corners then apply rule 2 or rule 3 as well as usual Manhattan distance then find minimum of them

def enhanced_Manhattan(puzzle, goalState):
    distance_sum_1 = 0
    distance_sum_2 = 0
    distance_sum_3 = 0
    distance_sum_4 = 0
    x,y = puzzle.shape
    shape = puzzle.shape
    empty_tile_position = np.argwhere(puzzle == 0)
    goal1 = goalState[0]
    goal2 = goalState[1]
    if np.array_equal(puzzle, goal1)or np.array_equal(puzzle, goal2):
        return 0
    # Checking if the empty tile is at the corner
    if is_corner(shape,empty_tile_position):
        # calculating normal manhattan distance & including rule 2, rule 3
        swappablePositions = generateSwappablePositions(empty_tile_position,shape)
        for i in range(x):
            for j in range(y):
                current_value = puzzle[i][j]
                if current_value != 0:
                    position_in_goal_state = np.argwhere(goal1 == current_value)
                    if [i,j] in swappablePositions:
                        cost1 = 1 + abs((position_in_goal_state[0][0] - empty_tile_position[0][0])) + abs((position_in_goal_state[0][1] - empty_tile_position[0][1]))
                        cost2 = current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                        minCost = min(cost1,cost2)
                        distance_sum_2 += minCost
                    else:
                        current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                        distance_sum_2 += current_distance
        for i in range(x):
            for j in range(y):
                current_value = puzzle[i][j]
                if current_value != 0:
                    position_in_goal_state = np.argwhere(goal2 == current_value)
                    if [i,j] in swappablePositions:
                        cost1 = 1 + abs((position_in_goal_state[0][0] - empty_tile_position[0][0])) + abs((position_in_goal_state[0][1] - empty_tile_position[0][1]))
                        cost2 = current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                        minCost = min(cost1,cost2)
                        distance_sum_3 += minCost
                    else:
                        current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                        distance_sum_3 += current_distance
        return min(distance_sum_2,distance_sum_3)
        
    else:
        # calculating normal manhattan distance
        for i in range(x):
            for j in range(y):
                current_value = puzzle[i][j]
                if current_value != 0:
                    position_in_goal_state = np.argwhere(goal1 == current_value)
                    current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                    distance_sum_1 += current_distance
        for i in range(x):
            for j in range(y):
                current_value = puzzle[i][j]
                if current_value != 0:
                    position_in_goal_state = np.argwhere(goal2 == current_value)
                    current_distance = abs((position_in_goal_state[0][0] - i)) + abs((position_in_goal_state[0][1] - j))
                    distance_sum_4 += current_distance
        return min(distance_sum_1,distance_sum_4)

       
def generateSwappablePositions(empty_tile_position, shape):
    position = []
    if empty_tile_position[0][0] == 0 and empty_tile_position[0][1] == 0:
        position.append([0+1,0+1])
        position.append([0,shape[1]-1])
        position.append([shape[0]-1,shape[1]-1])
        return position
    if empty_tile_position[0][0] == 0 and empty_tile_position[0][1] == shape[1]-1:
        position.append([0+1,shape[1]-1-1])
        position.append([0,0])
        position.append([shape[0]-1,0])
        return position
    if empty_tile_position[0][0] == shape[0]-1 and empty_tile_position[0][1] == 0:
        position.append([shape[0]-1-1,0+1])
        position.append([shape[0]-1,shape[1]-1])
        position.append([0,shape[1]-1])
        return position
    if empty_tile_position[0][0] == shape[0]-1 and empty_tile_position[0][1] == shape[1]-1:
        position.append([0,0])
        position.append([shape[0]-1-1,shape[1]-1-1])
        position.append([shape[0]-1,0])
        return position

In [6]:
# heuristic function 2
# simple heuristic which evaluate based on each row of the array and each column of the array, 
# eg. if an element is now in row 1 but it should be in row 2, cost +1 if an element is now in column 1 but should be in column 2 cost +1
# total cost = 1 + 1 = 2 


def rowColumnEvaluation(puzzle, goalState):
    h1 = 0
    h2 = 0
    x,y = puzzle.shape
    goal1 = goalState[0]
    goal2 = goalState[1]
    # Evaluating heuristic for goal1
    # Evaluating the rows
    for i in range(x):
        elementInThisRow = []
        for j in range(y):
             elementInThisRow.append(goal1[i][j])
        for k in range(y):
            if not puzzle[i][k] in elementInThisRow and puzzle[i][k] != 0:
                h1 += 1
    # Evaluating the columns
    for i in range(y):
        elementInThisColumn = []
        for j in range(x):
            elementInThisColumn.append(goal1[j][i])
        for k in range(x):
            if not puzzle[k][i] in elementInThisColumn and puzzle[k][i] != 0:
                h1 += 1
    # Evaluating heuristic for goal2
    for i in range(x):
        elementInThisRow = []
        for j in range(y):
             elementInThisRow.append(goal2[i][j])
        for k in range(y):
            if not puzzle[i][k] in elementInThisRow and puzzle[i][k] != 0:
                h2 += 1
    # Evaluating the columns
    for i in range(y):
        elementInThisColumn = []
        for j in range(x):
            elementInThisColumn.append(goal2[j][i])
        for k in range(x):
            if not puzzle[k][i] in elementInThisColumn and puzzle[k][i] != 0:
                h2 += 1
    return min(h1,h2)
            





In [8]:
arr = np.array([[1,2,3,4],[5,6,7,0]])
goal = [[[1,2,3,4],[5,6,7,0]],[[1,3,5,7],[2,4,6,0]]]
print(enhanced_Manhattan(arr,goal))

0


In [24]:
arr1 = np.array([[3,4,2,1],[5,6,7,0]])
goallist = [[[1,2,3,4],[5,6,7,0]],[[1,3,5,7],[2,4,6,0]]]

print(rowColumnEvaluation(arr1,goallist))

4
